In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling, pipeline
import os

os.environ["CUDA_VISIBLE_DEVICES"] = '1'

/home/27a_sul@lab.graphicon.ru/miniconda3/envs/nlp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "ai-forever/rugpt2large"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

tokenizer.add_special_tokens({'pad_token': '<|pad|>', 'bos_token': '<|startoftext|>', 'eos_token': '<|endoftext|>'})
model.resize_token_embeddings(len(tokenizer))

Embedding(50260, 1280)

In [ ]:
!gdown 18oCseBD3UpD2ode0TQqMHlcLMULNa1cZ

In [21]:
def load_dataset(file_path, tokenizer, block_size=256):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )
    return dataset

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

file_path = "aneki.txt"
train_dataset = load_dataset(file_path, tokenizer)

In [22]:
training_args = TrainingArguments(
    output_dir="./model_output",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=500,
    fp16=True,
    prediction_loss_only=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator
)

trainer.train()

Step,Training Loss
100,3.813700
200,2.999400
300,2.937600
400,2.835700
500,2.806000
600,2.804700
700,2.594200
800,2.547700
900,2.543400


TrainOutput(global_step=930, training_loss=2.8642012729439683, metrics={'train_runtime': 1833.1021, 'train_samples_per_second': 16.251, 'train_steps_per_second': 0.507, 'total_flos': 3.23771362443264e+16, 'train_loss': 2.8642012729439683, 'epoch': 2.996375352396295})

In [ ]:
trainer.save_model("./fine_tuned_rugpt2")
tokenizer.save_pretrained("./fine_tuned_rugpt2")

In [5]:
fine_tuned_model = GPT2LMHeadModel.from_pretrained("./fine_tuned_rugpt2")

fine_tuned_tokenizer = GPT2Tokenizer.from_pretrained("./fine_tuned_rugpt2")

In [7]:
generator = pipeline("text-generation", model=fine_tuned_model, tokenizer=fine_tuned_tokenizer, device=0)

num_samples = 5

result = generator(
    "<|startoftext|>",
    max_length=200,
    num_return_sequences=5,
    do_sample=True,
    temperature=1.0,
    top_k=50,
    top_p=0.9,
    use_cache=True
)

for i, sample in enumerate(result):
    print(f"\nSample {i + 1}:\n{sample['generated_text']}\n")
    print('#' * 120)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Sample 1:
<|startoftext|>- Что-то в последнее время мне вообще никто не нравится. Даже кот.- Может, тебе просто никто не нужен.
- Я хочу завести собаку.- А почему не кошку?- Кошки постоянно ходят.
- Если ты все время будешь ходить вокруг, вечно оглядываясь, люди подумают, что ты труслива.
Коротко о себе: мне 25 лет, я худенькая, стройненькая, с маленькой грудью и огромными глазами. У меня очень длинные и густые ресницы. А когда они падают на глаза, я просто ничего не могу с этим поделать.
- А почему ты сегодня в школу не пришла?- Папа, я уже большая девочка. Сама решу, когда и где мне быть...
Все девушки одинаковые. Просто у некоторых есть грудь, а у некоторых - мозг.
- Дорогая, как ты думаешь, если я заведу любовника, он будет называть меня своей принцессой?- Конечно!- Тогда я хочу завести

########################################################################################################################

Sample 2:
<|startoftext|>А почему только в России можно купить кота в мешк